In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dense

In [ ]:
input_size = 128
kernel_size = 3
maxpool_size = 2
batch_size = 4
epochs = 100

model = Sequential()
model.add(Conv2D(16,(kernel_size,kernel_size),input_shape=(input_size,input_size,1),activation ='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(maxpool_size, maxpool_size)))

model.add(Conv2D(32,(kernel_size,kernel_size),activation ='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(maxpool_size, maxpool_size)))

model.add(Conv2D(64, (kernel_size,kernel_size), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(maxpool_size, maxpool_size)))


model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(24,activation='softmax'))

model.compile( optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
train_dir = "D:/data/my_dataset/asl-dataset-gray-2/train/"
val_dir = "D:/data/my_dataset/asl-dataset-gray-2/val/"
test_dir = "D:/data/my_dataset/asl-dataset-gray-2/test/"

train_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=None,
    shear_range=0,
    zoom_range=0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1/255,
    preprocessing_function=None,
    data_format=None,
    dtype=None
)
gen = ImageDataGenerator(rescale=1/255)

training_set = gen.flow_from_directory(train_dir, target_size = (input_size, input_size), batch_size = batch_size , class_mode ='categorical', color_mode='grayscale', shuffle=True)
validation_set = gen.flow_from_directory(val_dir, target_size = (input_size, input_size), batch_size = batch_size , class_mode ='categorical', color_mode='grayscale', shuffle=True)
testing_set = gen.flow_from_directory(test_dir, target_size = (input_size, input_size), batch_size= batch_size, class_mode ='categorical', color_mode='grayscale')

In [ ]:
cb = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=5,verbose=1)
history = model.fit(
    training_set,
    steps_per_epoch= len(training_set)//batch_size,
    epochs= epochs,
    validation_data=validation_set,
    validation_steps= len(validation_set)//batch_size,
    callbacks=[cb]
)

In [ ]:
import matplotlib.pyplot as plt

loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
plt.plot(loss_train, 'r', label='Training accuracy')
plt.plot(loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

loss_train = history.history['loss']
loss_val = history.history['val_loss']
plt.plot(loss_train, 'r', label='Training loss')
plt.plot(loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

score = model.evaluate(testing_set)
for idx, metric in enumerate(model.metrics_names):
    print(metric, score[idx])

model.save("ai_model")